In [1]:
import awkward as ak
import numpy as np
import uproot as uproot
import matplotlib.pyplot as plt
import mplhep as hep
import vector as vec
from tqdm import tqdm
import ROOT
import math
'''
from utils import *
from utilsPCA import *
from plotting import *
from efficiency import *
from isolation import *
'''
import matplotlib as mpl

#mpl.rc('xtick', labelsize=16) 
#mpl.rc('ytick', labelsize=16) 
#mpl.rc('axes', labelsize=16, titlesize=20)
#mpl.rcParams["legend.title_fontsize"]=18

#plt.style.use(hep.style.CMS)
%matplotlib inline
lw=1.2

ROOT.enableJSVis()
    
import pickle



folders_1p9 = ["SinglePionTiming_1p9_100GeV", "SinglePionTiming_1p9_10GeV",
 "SinglePionTiming_1p9_15GeV", "SinglePionTiming_1p9_2GeV",
 "SinglePionTiming_1p9_30GeV", "SinglePionTiming_1p9_4GeV",
 "SinglePionTiming_1p9_50GeV", "SinglePionTiming_1p9_6GeV",
 "SinglePionTiming_1p9_8GeV"]


folders_2p2 = ["SinglePionTiming_2p2_100GeV", "SinglePionTiming_2p2_10GeV",
 "SinglePionTiming_2p2_15GeV", "SinglePionTiming_2p2_2GeV",
 "SinglePionTiming_2p2_30GeV", "SinglePionTiming_2p2_4GeV",
 "SinglePionTiming_2p2_50GeV", "SinglePionTiming_2p2_6GeV",
 "SinglePionTiming_2p2_8GeV"]

dofit=False
pt=True

Welcome to JupyROOT 6.30/04


In [2]:

def compute_residuals_energiesALL(file_name):
    
    open_file = uproot.open(file_name)
    f_simTICLCandidate = open_file["ticlDumper/simTICLCandidate;1"]
    f_candidates = open_file["ticlDumper/candidates;1"]
    f_trackstersTICL=open_file["ticlDumper/trackstersTiclCandidate;1"]
    f_associations = open_file["ticlDumper/associations;1"]
    candidate_time = f_candidates["candidate_time"].array()
    candidate_timeErr= f_candidates["candidate_timeErr"].array()
    
    candidate_time_MTD = f_candidates["candidate_time_MTD"].array()
    candidate_time_MTDErr= f_candidates["candidate_time_MTD_err"].array()
    
    candidate_energy= f_candidates["candidate_energy"].array()
    candidate_px= f_candidates["candidate_px"].array()
    candidate_py= f_candidates["candidate_py"].array()
    trackster_energy=f_trackstersTICL["raw_energy"].array()#f_candidates["candidate_raw_energy"].array()
    
    print(f_associations["ticlCandidate_simToReco_CP"].array()[0])
    print(len(f_associations["ticlCandidate_simToReco_CP"].array()[0]))
    assoc=f_associations["ticlCandidate_simToReco_CP"].array()
    assoc_shE=f_associations["ticlCandidate_simToReco_CP_sharedE"].array()

    #print(len(assoc[1]),len(candidate_energy[1]))

    #print(candidate_energy[0][6])
    x=0
    #print(sum([len(y) for y in candidate_energy]))
    ncandidates=sum([len(y) for y in candidate_energy])
    counter=0
    listvalid=[]
    
    for i in tqdm(range(len(assoc))):
        listvalid.append([])
        for j in range(len(assoc[i])):
            for k in range(len(assoc[i][j])):
                #print(i,j,k)
                #if assoc[i][j][k]==3 and i==118:
                #    print("--" , trackster_energy[i][assoc[i][j][k]], "sh:",assoc_shE[i][j][k])
                
                if assoc_shE[i][j][k]>trackster_energy[i][assoc[i][j][k]]*0.5:
                    listvalid[i].append(assoc[i][j][k])
                    if assoc_shE[i][j][k]/trackster_energy[i][assoc[i][j][k]]>1.001:
                        print("sus", i,j,k,assoc[i][j][k])
                    #print("top")
                    #print(assoc_shE[i][j][k],candidate_energy[i][assoc[i][j][k]])
                    counter+=1
                #candidate_energy[i][j][assoc[i][j][k]]

    print("efficient:", counter,"inefficient linking or trackless:", ncandidates-counter,"efficiency:",counter/ncandidates)
    print("trackless candidates:")
    
    #print("")
    
    
    tracksters_in_candidate=f_candidates["tracksters_in_candidate"].array()
    
    
    simcandidate_time = f_simTICLCandidate["simTICLCandidate_time"].array()
    simcandidate_charge=f_simTICLCandidate["simTICLCandidate_charge"].array()
    
    candidate_charge=f_candidates["candidate_charge"].array()
    
    ticl_residuals={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    ticl_energies={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    ticl_pt={"avg":[],
             "hgcal":[],
             "mtd":[]}
    
    ticl_isCharged={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    trackless=0
    for i in tqdm(range(int(len(simcandidate_time)))):
        #if ak.max(assoc[i])>len(candidate_time[i]):
        #if len(candidate_time[i])>len(trackster_energy[i]):
        #        print(i, len(candidate_time[i]),len(candidate_energy[i]),len(trackster_energy[i]),ak.max(assoc[i]))
        #        print(trackster_energy[i])
        for j in range(len(candidate_time[i])):
            if (len(tracksters_in_candidate[i][j])>1):
                print("ATTENZIONE")
            t_MTD=False
            t_HGCal=False
            time_avg=0
            time_avgErr=0
            trck_in_cand=tracksters_in_candidate[i][j]
            if (len(trck_in_cand)>0):
                if(len(trck_in_cand)>1): 
                    print("---- Warning: more than 1 trackster in candidate")
                if trck_in_cand[0] in listvalid[i]:
                    if (candidate_timeErr[i][j]>0):
                        t_HGCal=True 
                        ticl_residuals["hgcal"].append(simcandidate_time[i][0]-candidate_time[i][j])
                        ticl_energies["hgcal"].append(candidate_energy[i][j])
                        ticl_pt["hgcal"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["hgcal"].append(not candidate_charge[i][j]==0)
                        time_avg=candidate_time[i][j]
                        time_avgErr=candidate_timeErr[i][j]

                    if(candidate_time_MTDErr[i][j]>0):
                        t_MTD=True
                        MTD_time=candidate_time_MTD[i][j]
                        ticl_residuals["mtd"].append(simcandidate_time[i][0]-MTD_time)
                        ticl_energies["mtd"].append(candidate_energy[i][j])
                        ticl_pt["mtd"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["mtd"].append(not candidate_charge[i][j]==0)
                        if(t_HGCal):
                            inv_hgcal_err=1./(time_avgErr**2)
                            inv_mtd_err=1./(candidate_time_MTDErr[i][j]**2)
                            time_avg=(candidate_time[i][j]*inv_hgcal_err+MTD_time*inv_mtd_err)/(inv_mtd_err+inv_hgcal_err)

                        else:
                            time_avg=MTD_time

                    if(t_MTD or t_HGCal):
                        ticl_residuals["avg"].append(simcandidate_time[i][0]-time_avg)
                        ticl_energies["avg"].append(candidate_energy[i][j])
                        ticl_pt["avg"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["avg"].append(not candidate_charge[i][j]==0)
            else:
                trackless+=1
            
            eff_track=(ncandidates-trackless)/ncandidates
            eff_link = counter/(ncandidates-trackless)
                        
    return ticl_residuals, ticl_energies, ticl_pt, ticl_isCharged,eff_track,eff_link

In [3]:
def pdf(x, par):
    A, mean, sigmaL, alphaL, sigmaR, alphaR = par

    x = np.asarray(x)
    if x.ndim == 0:
        x = np.array([x])

    answer = np.zeros(x.shape)

    left = x < mean
    right = np.logical_not(left)

    top = np.square(x - mean)

    bottomL = 2 * (sigmaL * sigmaL + alphaL * top[left])
    bottomR = 2 * (sigmaR * sigmaR + alphaR * top[right])

    answer[left] = A * np.exp(-np.divide(top[left], bottomL))
    answer[right] = A * np.exp(-np.divide(top[right], bottomR))
    return answer


def cruijff(x, params):
    A = params[0]
    m = params[1]
    sigma_left = params[2]
    sigma_right = params[3]
    alpha_left = params[4]
    alpha_right = params[5]
    
    t = (x[0] - m) 
    if t < 0:
        t = (x[0] - m)/sigma_left 
        return A*math.exp(-0.5 * t * t / (1 + 0.5*alpha_left * t * t))
    else:
        t = (x[0] - m)/sigma_right
        return A*math.exp(-0.5 * t * t / (1 + 0.5*alpha_right * t * t))
    
def res_E_fit(x,par):
    S=par[0]
    N=par[1]
    C=par[2]
    
    return math.sqrt( (S/math.sqrt(x[0]))**2+(N/x[0])**2+C**2)

    

# Funz per fittare

# Load selected data >

In [4]:
file = open('data_assoc/2p2.pickle', 'rb')
dict_data = pickle.load(file)
file.close()


In [5]:


from multiprocessing import Process, Queue

import re


def resolution(folder,tag,dict_resolutions,Q):
    
    
    t_res_all, energies_all, pt_all, isCharged_all,ticl_w_true_trackster,ticl_w_true_track,ticl_w_true_pid,ticl_w_tracksters=dict_data[folder]
   
    nbins=100
    match = re.search(r'(\d+)GeV', folder)
    


    histo={"avg":ROOT.TH1F("combined","Res Combination p_{T}="+match.group(1)+" GeV/c;Res [ns];Counts",nbins,-0.1,0.1),
          "hgcal":ROOT.TH1F("hgcal","Res HGCal;Res [ns];Counts",nbins,-0.1,0.1),
          "mtd":ROOT.TH1F("mtd","Res MTD;Res [ns];Counts",nbins,-0.1,0.1)}
    
    #f_gaus=ROOT.TF1("f1","gaus(0)",-10.,10.)
    f_cruijff=ROOT.TF1("f2",cruijff,-10,10.,6)

    for key in t_res_all:
        for i in range(len(t_res_all[key])):
            if isCharged_all[key][i]:
                histo[key].Fill(t_res_all[key][i])



    ROOT.gStyle.SetOptFit(True)

    #mg=ROOT.TMultiGraph()

    c1=ROOT.TCanvas()
    c1.cd()
    histo["avg"].SetLineColor(ROOT.kMagenta+2)

    histo["hgcal"].SetLineColor(ROOT.kCyan+2)


    histo["mtd"].SetLineColor(ROOT.kSpring+4)

    [histo[key].SetLineWidth(2) for key in histo]
    [histo[key].Sumw2() for key in histo]
    [histo[key].Draw("SAME E") for key in histo]
    
    c1.BuildLegend()
    
    f_cruijff.SetParNames("A","#mu","#sigma_L","#sigma_R","#alpha_L","#alpha_R")
    f_cruijff.SetParameters(3000,0.0019,0.0173,0.0173,0,0)
    f_cruijff.SetParLimits(2,0.01,10)
    f_cruijff.SetParLimits(3,0.01,10)
    f_cruijff.SetParLimits(4,0.00001,10)
    f_cruijff.SetParLimits(5,0.00001,10)
    
    match = re.search(r'(\d+)GeV', folder)
    
    
    
    
    
    resolutions={}
    resolution_errs={}
    
    for key in t_res_all:
        if(int(match.group(1))==2):
            print(2)
            f_cruijff.SetParameters(200,-0.0007,0.0224223,0.0224223,0.01,0.01)
            f_cruijff.FixParameter(4,0.0001)
            f_cruijff.FixParameter(5,0.0001)
            fitresult=histo[key].Fit(f_cruijff,"LN","",-0.03,0.03)
            f_cruijff.SetParLimits(2,0.01,10)
            f_cruijff.SetParLimits(3,0.01,10)
            f_cruijff.ReleaseParameter(4)
            f_cruijff.ReleaseParameter(5)
            #f_cruijff.FixParameter(4,0.0001)
            #f_cruijff.FixParameter(5,0.0001)
        if(int(match.group(1))==4 or int(match.group(1))==6 or int(match.group(1))==8 or int(match.group(1))==15 or int(match.group(1))==10):
            f_cruijff.SetParameters(200,-0.0007,0.0224223,0.0224223,0.01,0.01)
            f_cruijff.FixParameter(4,0.0001)
            f_cruijff.FixParameter(5,0.0001)
            fitresult=histo[key].Fit(f_cruijff,"LN","",-0.03,0.03)
            f_cruijff.SetParLimits(2,0.01,10)
            f_cruijff.SetParLimits(3,0.01,10)
            f_cruijff.ReleaseParameter(4)
            f_cruijff.ReleaseParameter(5)
            #f_cruijff.FixParameter(4,0.0001)
            #f_cruijff.FixParameter(5,0.0001)
        
        print(folder)
        f_cruijff.ReleaseParameter(4)
        f_cruijff.ReleaseParameter(5)
        fitresult=histo[key].Fit(f_cruijff,"MLS+","SAME",-0.1,0.1)
        cov_matrix=fitresult.GetCovarianceMatrix()
        params=f_cruijff.GetParameters()
        sigma_c=(params[2]+params[3])/2

        resolutions[key]=sigma_c
        resolution_errs[key]=math.sqrt(0.25*(cov_matrix[2][2]+cov_matrix[3][3])+ 0.5*cov_matrix[2][3])
    
    
    dict_resolutions[int(match.group(1))]=[resolutions,resolution_errs]
    Q.put(dict_resolutions)
    if "1p9" in folder:
        c1.SaveAs("PlotAssoc/H"+str(tag)+".pdf")
        
    if "2p2" in folder:
        c1.SaveAs("PlotAssoc/H"+str(tag)+".pdf")
        
    c1.SaveAs("H"+str(tag)+".pdf")
    

# Fit for 1.9

In [6]:
procs=[]

q=Queue()
dict_resolutions={}
res={}
    
numeri_GeV=[]
for folder in folders_1p9:
    match = re.search(r'(\d+)GeV', folder)
    if match:
        numeri_GeV.append(int(match.group(1)))
    p=Process(target=resolution,args=(folder,folder,dict_resolutions,q))
    procs.append(p)
    p.start()
    p.join()
    #t_resolution,t_resolution_err=resolution(folder,folder,dict_resolutions)



    

    
for job in procs:
    job.join()

for i in range(len(folders_1p9)):
    dict_resolutions.update(q.get())
    
print(dict_resolutions)

    
    
import pickle


with open('data_assoc/res1p9.pickle', 'wb') as handle:
    pickle.dump(dict_resolutions, handle, protocol=pickle.HIGHEST_PROTOCOL)



SinglePionTiming_1p9_100GeV
SinglePionTiming_1p9_100GeV
SinglePionTiming_1p9_100GeV
SinglePionTiming_1p9_10GeV
SinglePionTiming_1p9_10GeV
SinglePionTiming_1p9_10GeV
SinglePionTiming_1p9_15GeV
SinglePionTiming_1p9_15GeV
SinglePionTiming_1p9_15GeV
2
SinglePionTiming_1p9_2GeV
2
SinglePionTiming_1p9_2GeV
2
SinglePionTiming_1p9_2GeV
SinglePionTiming_1p9_30GeV
SinglePionTiming_1p9_30GeV
SinglePionTiming_1p9_30GeV
SinglePionTiming_1p9_4GeV
SinglePionTiming_1p9_4GeV
SinglePionTiming_1p9_4GeV
SinglePionTiming_1p9_50GeV
SinglePionTiming_1p9_50GeV
SinglePionTiming_1p9_50GeV
SinglePionTiming_1p9_6GeV
SinglePionTiming_1p9_6GeV
SinglePionTiming_1p9_6GeV
SinglePionTiming_1p9_8GeV
SinglePionTiming_1p9_8GeV
SinglePionTiming_1p9_8GeV
{100: [{'avg': 0.012788840558168636, 'hgcal': 0.014805722842011617, 'mtd': 0.024821418231865934}, {'avg': 8.943884667758687e-05, 'hgcal': 0.00010901715542613524, 'mtd': 0.00023599260134570332}], 10: [{'avg': 0.014751383357628874, 'hgcal': 0.018300453257846155, 'mtd': 0.0250

Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_1p9_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_1p9_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_1p9_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_1p9_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_1p9_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_1p9_4GeV.pdf has been created
Info in <TCanvas:

# Multiplot for 1.9

In [7]:


file = open('data_assoc/res1p9.pickle', 'rb')
dict_resolutions = pickle.load(file)
file.close()



g_dict={"avg":ROOT.TGraphErrors(),
        "hgcal":ROOT.TGraphErrors(),
        "mtd":ROOT.TGraphErrors()}

print(dict_resolutions)
print(dict_resolutions[2])
counter=0
for enkey in dict_resolutions.keys():
    for typekey in dict_resolutions[enkey][0]:
        if pt:
            g_dict[typekey].AddPoint(enkey,dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        else:
            g_dict[typekey].AddPoint(enkey*math.cosh(1.9),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        
        g_dict[typekey].SetPointError(counter,0,dict_resolutions[enkey][1][typekey])
    counter=counter+1
cmg=ROOT.TCanvas()
cmg.cd()
mg=ROOT.TMultiGraph()


g_dict["avg"].SetLineColor(ROOT.kMagenta+2)
g_dict["avg"].SetMarkerColor(ROOT.kMagenta+2)
g_dict["avg"].SetMarkerStyle(22)
g_dict["avg"].SetTitle("Combination")

g_dict["hgcal"].SetLineColor(ROOT.kCyan+2)
g_dict["hgcal"].SetMarkerColor(ROOT.kCyan+2)
g_dict["hgcal"].SetMarkerStyle(20)
g_dict["hgcal"].SetTitle("HGCal time")


g_dict["mtd"].SetLineColor(ROOT.kSpring+4)
g_dict["mtd"].SetMarkerColor(ROOT.kSpring+4)
g_dict["mtd"].SetMarkerStyle(34)
g_dict["mtd"].SetTitle("MTD time")
#[g_dict[x].SetTitle(str(x)) for x in g_dict.keys()]

#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]


f_res_E=ROOT.TF1("fres",res_E_fit,0.001,600.,3)#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]
f_res_E.SetNpx(1000)
f_res_E.SetParameters(1.,1.,0.016)
f_res_E.SetParNames("S","N","C")


ROOT.gStyle.SetOptFit(True)
if dofit:
    g_dict["hgcal"].Fit(f_res_E,"r")

[mg.Add(g_dict[x]) for x in g_dict.keys()]

if pt:
    mg.SetTitle("Time resolution;p_{T} [GeV/c]; Resolution [ns]")

else:
    mg.SetTitle("Time resolution;E [GeV]; Resolution [ns]")
mg.Draw("AP")

#cmg.BuildLegend()
legend=ROOT.TLegend(0.2,0.8,0.4,0.7)

legend.AddEntry(g_dict["avg"])
legend.AddEntry(g_dict["mtd"])
legend.AddEntry(g_dict["hgcal"])
legend.Draw()

text = ROOT.TLatex()
text.SetNDC(1)  # Imposta le coordinate in termini di frazione del canvas (0-1)
text.SetTextSize(0.04)  # Imposta la dimensione del testo
text.SetTextAlign(31)  # Allinea il testo a destra rispetto alla posizione specificata
text.DrawLatex(0.9, 0.92, "#eta=1.9")  # (x, y, testo) posizione nell'angolo in alto a destra
#mg.SetMinimum(0.)
#mg.SetMaximum(0.035)



mg.SetMinimum(0.)
mg.SetMaximum(0.035)

cmg.Draw()
if dofit:
    if pt:
        cmg.SaveAs("PlotAssoc/Resolution_1p9_pT_fit.pdf")
    else:
        cmg.SaveAs("PlotAssoc/Resolution_1p9_E_fit.pdf")
else:
    if pt:
        cmg.SaveAs("PlotAssoc/Resolution_1p9_pT.pdf")
    else: 
        cmg.SaveAs("PlotAssoc/Resolution_1p9_E.pdf")
    




{100: [{'avg': 0.012788840558168636, 'hgcal': 0.014805722842011617, 'mtd': 0.024821418231865934}, {'avg': 8.943884667758687e-05, 'hgcal': 0.00010901715542613524, 'mtd': 0.00023599260134570332}], 10: [{'avg': 0.014751383357628874, 'hgcal': 0.018300453257846155, 'mtd': 0.02505013029587001}, {'avg': 0.0001583377285222746, 'hgcal': 0.00021233482869263974, 'mtd': 0.00033910404655911}], 15: [{'avg': 0.014164111952589629, 'hgcal': 0.017572259666111964, 'mtd': 0.024140354100331968}, {'avg': 0.00012655866666485784, 'hgcal': 0.00016972634009634286, 'mtd': 0.00027612976059235896}], 2: [{'avg': 0.018206943361806567, 'hgcal': 0.023425778393594606, 'mtd': 0.023808843918504817}, {'avg': 0.0005247349211989733, 'hgcal': 0.0008712642018004144, 'mtd': 0.0007194196192366672}], 30: [{'avg': 0.013617131059101513, 'hgcal': 0.016195564267928814, 'mtd': 0.024461841163088974}, {'avg': 0.00010295531707246519, 'hgcal': 0.00013149463353726221, 'mtd': 0.0002491421063040542}], 4: [{'avg': 0.015287613311637421, 'hgca

Info in <TCanvas::Print>: pdf file PlotAssoc/Resolution_1p9_pT.pdf has been created


# Fit 2p2

In [8]:
procs=[]

q=Queue()
dict_resolutions={}
res={}
    
numeri_GeV=[]

for folder in folders_2p2:
    match = re.search(r'(\d+)GeV', folder)
    if match:
        numeri_GeV.append(int(match.group(1)))
    p=Process(target=resolution,args=(folder,folder,dict_resolutions,q))
    procs.append(p)
    p.start()
    p.join()
    #t_resolution,t_resolution_err=resolution(folder,folder,dict_resolutions)



    

    
for job in procs:
    job.join()

for i in range(len(folders_1p9)):
    dict_resolutions.update(q.get())
    
print(dict_resolutions)

    
    
import pickle


with open('data_assoc/res2p2.pickle', 'wb') as handle:
    pickle.dump(dict_resolutions, handle, protocol=pickle.HIGHEST_PROTOCOL)



SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_15GeV
SinglePionTiming_2p2_15GeV
SinglePionTiming_2p2_15GeV
2
SinglePionTiming_2p2_2GeV
2
SinglePionTiming_2p2_2GeV
2
SinglePionTiming_2p2_2GeV
SinglePionTiming_2p2_30GeV
SinglePionTiming_2p2_30GeV
SinglePionTiming_2p2_30GeV
SinglePionTiming_2p2_4GeV
SinglePionTiming_2p2_4GeV
SinglePionTiming_2p2_4GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_8GeV
SinglePionTiming_2p2_8GeV
SinglePionTiming_2p2_8GeV
{100: [{'avg': 0.012637688079611513, 'hgcal': 0.01472073732147569, 'mtd': 0.024496972410532156}, {'avg': 8.832430997641405e-05, 'hgcal': 0.00010781963590589343, 'mtd': 0.0002342834675041325}], 10: [{'avg': 0.015781413962439823, 'hgcal': 0.02010805313717981, 'mtd': 0.0242794

Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_4GeV.pdf has been created
Info in <TCanvas:

# Multiplot 2p2

In [9]:


file = open('data_assoc/res2p2.pickle', 'rb')
dict_resolutions = pickle.load(file)
file.close()



g_dict2p2={"avg":ROOT.TGraphErrors(),
        "hgcal":ROOT.TGraphErrors(),
        "mtd":ROOT.TGraphErrors()}

print(dict_resolutions)
print(dict_resolutions[2])
counter=0
for enkey in dict_resolutions.keys():
    for typekey in dict_resolutions[enkey][0]:
        
        #g_dict2p2[typekey].AddPoint(enkey*math.cosh(2.2),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        if pt:
            g_dict2p2[typekey].AddPoint(enkey,dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        else:
            g_dict2p2[typekey].AddPoint(enkey*math.cosh(2.2),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        
        g_dict2p2[typekey].SetPointError(counter,0,dict_resolutions[enkey][1][typekey])
    counter=counter+1
cmg2p2=ROOT.TCanvas()
cmg2p2.cd()
mg2p2=ROOT.TMultiGraph()


g_dict2p2["avg"].SetLineColor(ROOT.kMagenta+2)
g_dict2p2["avg"].SetMarkerColor(ROOT.kMagenta+2)
g_dict2p2["avg"].SetMarkerStyle(22)
g_dict2p2["avg"].SetTitle("Combination")

g_dict2p2["hgcal"].SetLineColor(ROOT.kCyan+2)
g_dict2p2["hgcal"].SetMarkerColor(ROOT.kCyan+2)
g_dict2p2["hgcal"].SetMarkerStyle(20)
g_dict2p2["hgcal"].SetTitle("HGCal time")


g_dict2p2["mtd"].SetLineColor(ROOT.kSpring+4)
g_dict2p2["mtd"].SetMarkerColor(ROOT.kSpring+4)
g_dict2p2["mtd"].SetMarkerStyle(34)
g_dict2p2["mtd"].SetTitle("MTD time")
#[g_dict[x].SetTitle(str(x)) for x in g_dict.keys()]

#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]


f_res_E.SetParameters(1.,1.,0.016)
f_res_E.SetParNames("S","N","C")


ROOT.gStyle.SetOptFit(True)
if dofit:
    g_dict2p2["hgcal"].Fit(f_res_E,"r")

[mg2p2.Add(g_dict2p2[x]) for x in g_dict2p2.keys()]
if pt:
    mg2p2.SetTitle("Time resolution;p_{T} [GeV/c]; Resolution [ns]")
else:
    mg2p2.SetTitle("Time resolution;E [GeV]; Resolution [ns]")
    
mg2p2.Draw("AP")

#cmg.BuildLegend()
legend=ROOT.TLegend(0.2,0.8,0.4,0.7)

legend.AddEntry(g_dict["avg"])
legend.AddEntry(g_dict["mtd"])
legend.AddEntry(g_dict["hgcal"])
legend.Draw()

text = ROOT.TLatex()
text.SetNDC(1)  # Imposta le coordinate in termini di frazione del canvas (0-1)
text.SetTextSize(0.04)  # Imposta la dimensione del testo
text.SetTextAlign(31)  # Allinea il testo a destra rispetto alla posizione specificata
text.DrawLatex(0.9, 0.92, "#eta=2.2")  # (x, y, testo) posizione nell'angolo in alto a destra
#mg.SetMinimum(0.)
#mg.SetMaximum(0.035)



mg2p2.SetMinimum(0.)
mg2p2.SetMaximum(0.035)

cmg2p2.Draw()



if dofit:
    if pt:
        cmg2p2.SaveAs("PlotAssoc/Resolution_2p2_pT_fit.pdf")
    else:
        cmg2p2.SaveAs("PlotAssoc/Resolution_2p2_E_fit.pdf")
    
else:
    if pt:
        cmg2p2.SaveAs("PlotAssoc/Resolution_2p2_pT.pdf")
    else:
        cmg2p2.SaveAs("PlotAssoc/Resolution_2p2_E.pdf")



{100: [{'avg': 0.012637688079611513, 'hgcal': 0.01472073732147569, 'mtd': 0.024496972410532156}, {'avg': 8.832430997641405e-05, 'hgcal': 0.00010781963590589343, 'mtd': 0.0002342834675041325}], 10: [{'avg': 0.015781413962439823, 'hgcal': 0.02010805313717981, 'mtd': 0.024279448253567152}, {'avg': 0.00015930211807013622, 'hgcal': 0.00022643276426015052, 'mtd': 0.0003087873881615911}], 15: [{'avg': 0.014731235607490229, 'hgcal': 0.01852317823103887, 'mtd': 0.02462375232963919}, {'avg': 0.00013277675515215162, 'hgcal': 0.00018246259996961697, 'mtd': 0.00028936839223444184}], 2: [{'avg': 0.020685416302517193, 'hgcal': 0.03111010344193105, 'mtd': 0.02234725964887236}, {'avg': 0.0004937683702784713, 'hgcal': 0.001435468976008114, 'mtd': 0.0005703175363773902}], 30: [{'avg': 0.013823400762448448, 'hgcal': 0.016960860638048546, 'mtd': 0.024880408856895055}, {'avg': 0.00010901392422943869, 'hgcal': 0.0001426215642138887, 'mtd': 0.0002615818755667946}], 4: [{'avg': 0.017787859999904447, 'hgcal': 0

Info in <TCanvas::Print>: pdf file PlotAssoc/Resolution_2p2_pT.pdf has been created


 # HGCal comparison

In [10]:


cmg1=ROOT.TCanvas()
cmg1.cd()
mg1=ROOT.TMultiGraph()

g_dict["hgcal"].SetTitle("HGCal, #eta=1.9")
g_dict["avg"].SetTitle("Combined, #eta=1.9")

mg1.Add(g_dict["hgcal"])
mg1.Add(g_dict["avg"])

g_dict2p2["hgcal"].SetLineColor(ROOT.kBlue+2)
g_dict2p2["hgcal"].SetMarkerColor(ROOT.kBlue+2)
g_dict2p2["hgcal"].SetTitle("HGCal, #eta=2.2")

g_dict2p2["avg"].SetLineColor(ROOT.kOrange+10)
g_dict2p2["avg"].SetMarkerColor(ROOT.kOrange+10)
g_dict2p2["avg"].SetTitle("Combined, #eta=2.2")




mg1.Add(g_dict2p2["hgcal"])
mg1.Add(g_dict2p2["avg"])

legendcmp=ROOT.TLegend(0.2,0.8,0.4,0.65)
legendcmp.AddEntry(g_dict["avg"])
legendcmp.AddEntry(g_dict["hgcal"])
legendcmp.AddEntry(g_dict2p2["avg"])
legendcmp.AddEntry(g_dict2p2["hgcal"])
if pt:
    mg1.SetTitle("Time resolution;p_{T} [GeV/c]; Resolution [ns]")
else:
    mg1.SetTitle("Time resolution;E [GeV]; Resolution [ns]")

mg1.SetMinimum(0.)
mg1.SetMaximum(0.040)
mg1.Draw("AP")

legendcmp.Draw()


#cmg1.Draw()
if dofit:
    if pt:
        
        cmg1.SaveAs("PlotAssoc/Comb_1p9_2p2_pT_fit.pdf")
    else:
        cmg1.SaveAs("PlotAssoc/Comb_1p9_2p2_E_fit.pdf")
else:
    if pt:
        cmg1.SaveAs("PlotAssoc/Comb_1p9_2p2_pT.pdf")
    else:
        cmg1.SaveAs("PlotAssoc/Comb_1p9_2p2_E.pdf")
    

Info in <TCanvas::Print>: pdf file PlotAssoc/Comb_1p9_2p2_pT.pdf has been created
